In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('/content/jokes-data.csv')

In [5]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163327 sha256=eb7db2464156b030058e9384791fb652df4113e4f2b7797b43b5973e0d7fd2a6
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [6]:
data.head()

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
1,16144_109,16144,109,5.094
2,23098_6,23098,6,-6.438
3,14273_86,14273,86,4.406
4,18419_134,18419,134,9.375


In [7]:
data.isna().sum()

id         0
user_id    0
joke_id    0
Rating     0
dtype: int64

In [8]:
data.dtypes

id          object
user_id      int64
joke_id      int64
Rating     float64
dtype: object

In [9]:
from surprise import Reader, Dataset, KNNBasic, SVD

In [10]:
data.Rating.unique()

array([  2.75 ,   5.094,  -6.438,   4.406,   9.375,  -1.781,   6.25 ,
        -2.906,  -5.75 ,   1.656,   0.688,   1.312,   6.031,   9.781,
         8.75 ,   0.156,   3.562,   2.938,   3.75 ,  -1.906,   3.469,
         7.   ,   4.656,   1.719,   6.219,  -8.25 ,   5.188,  -2.281,
         1.406,   2.906,   3.625,   6.719,   5.656,   8.562,   2.156,
        -5.344,   3.25 ,   1.   ,   7.375,   8.031,   6.125,   7.469,
         2.125,   4.062,  -8.   ,  -2.75 ,   9.406,   8.875,   4.5  ,
        -3.188,  -6.875,   2.719,   0.562,   2.969,   8.375,  -4.906,
         8.594,  -7.469,   0.25 ,   6.406,   5.125,   3.156,   0.312,
         0.969,  -3.969,   5.438,  -0.031,  -1.969,   4.688,   2.781,
         0.875,  -9.625,  -2.188,  -3.062,   8.781,   6.938,  -7.406,
         9.281,  -0.5  ,   1.25 ,   5.031,   3.375,   4.375,   6.875,
         6.688,   8.531,   5.594,   7.75 ,   8.281,  -0.531,   9.5  ,
         6.812,  -4.125,   1.875,   3.688,   4.438,  -5.5  ,  -1.406,
         4.031,  -0.

In [11]:
data.Rating.min()

-10.0

In [12]:
data.Rating.max()

10.0

In [13]:

old_min = -10
old_max = 10
new_min = 1
new_max = 5
scaling_factor = (new_max - new_min) / (old_max - old_min)

def rating_converter(original_rating):
  global scaling_factor
  new_rating = (original_rating - old_min) * scaling_factor + new_min
  return new_rating

data['Scaled_Rating'] = data.Rating.apply(rating_converter)

In [14]:
data

,id,user_id,joke_id,Rating,Scaled_Rating
0,31030_110,31030,110,2.750,3.5500
1,16144_109,16144,109,5.094,4.0188
2,23098_6,23098,6,-6.438,1.7124
3,14273_86,14273,86,4.406,3.8812
4,18419_134,18419,134,9.375,4.8750
...,...,...,...,...,...
341116,30189_4,30189,4,1.969,3.3938
341117,10586_117,10586,117,2.969,3.5938
341118,28697_137,28697,137,9.031,4.8062
341119,1564_6,1564,6,7.469,4.4938


In [15]:
data.drop('Rating', axis=1, inplace=True)

In [16]:
data

,id,user_id,joke_id,Scaled_Rating
0,31030_110,31030,110,3.5500
1,16144_109,16144,109,4.0188
2,23098_6,23098,6,1.7124
3,14273_86,14273,86,3.8812
4,18419_134,18419,134,4.8750
...,...,...,...,...
341116,30189_4,30189,4,3.3938
341117,10586_117,10586,117,3.5938
341118,28697_137,28697,137,4.8062
341119,1564_6,1564,6,4.4938


In [17]:
data.drop('id', axis=1, inplace=True)

In [19]:
reader = Reader(rating_scale=(1,5))

In [21]:
surprise_dataset = Dataset.load_from_df(data, reader)

In [22]:
algo = SVD()

In [23]:
from surprise.model_selection import cross_validate

In [24]:
cross_validate(algo, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.88920314, 0.89260806, 0.89554531, 0.88852516, 0.89193066]),
 'fit_time': (5.959188461303711,
  6.2549004554748535,
  7.400691032409668,
  7.7609593868255615,
  6.035923957824707),
 'test_time': (0.6653482913970947,
  0.6508429050445557,
  0.7093486785888672,
  0.6433920860290527,
  0.583010196685791)}

In [25]:
from surprise import NMF

In [26]:
algo2 = NMF()

In [27]:
cross_validate(algo2, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.94872498, 0.95769714, 0.95871194, 0.95527818, 0.96502444]),
 'fit_time': (11.839819192886353,
  11.577639102935791,
  11.824709415435791,
  12.75580620765686,
  11.748380422592163),
 'test_time': (0.5907652378082275,
  0.6442511081695557,
  0.3718559741973877,
  0.6026709079742432,
  1.0996873378753662)}

In [28]:
from surprise import SlopeOne
algo3 = SlopeOne()

In [29]:
cross_validate(algo3, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.89923195, 0.89159381, 0.8973432 , 0.89757957, 0.89725063]),
 'fit_time': (0.93904709815979,
  0.741997241973877,
  0.750544548034668,
  0.7551052570343018,
  1.1469089984893799),
 'test_time': (2.0324771404266357,
  1.0719232559204102,
  1.4034662246704102,
  1.3627033233642578,
  1.66066575050354)}

In [30]:
from surprise import BaselineOnly

In [31]:
algo4 = BaselineOnly()
cross_validate(algo3, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.89484717, 0.89331253, 0.89927043, 0.89255026, 0.8984754 ]),
 'fit_time': (0.8053524494171143,
  0.7631537914276123,
  0.7467367649078369,
  0.7513985633850098,
  0.7661342620849609),
 'test_time': (1.4041330814361572,
  1.9758124351501465,
  1.409148931503296,
  1.4650399684906006,
  1.3806896209716797)}

In [32]:

from surprise.model_selection import GridSearchCV
param_grid = {
    'n_factors': [10, 20, 30],
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005, 0.01]
}

In [ ]:
#Splitting

In [33]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_dataset, test_size=0.2)

In [34]:
best_rmse = float('inf')
best_params = {}

for n_factors in param_grid['n_factors']:
    for n_epochs in param_grid['n_epochs']:
        for lr_all in param_grid['lr_all']:
            algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, verbose=False)
            results = cross_validate(algo, surprise_dataset, measures=['RMSE'], cv=5, verbose=False)
            mean_rmse = results['test_rmse'].mean()
            if mean_rmse < best_rmse:
                best_rmse = mean_rmse
                best_params = {'n_factors': n_factors, 'n_epochs': n_epochs, 'lr_all': lr_all}

print("Best Hyperparameters:", best_params)
print("Best RMSE:", best_rmse)

Best Hyperparameters: {'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.005}
Best RMSE: 0.8799491348529249


In [35]:
best_model = SVD(n_factors = 20, n_epochs=20, lr_all=0.005)

In [36]:
best_model.fit(trainset)

In [37]:
predictions = best_model.test(testset)

In [38]:
from surprise import accuracy
rmse = accuracy.rmse(predictions)

RMSE: 0.8835
